In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2023-12-21 13:27:41.046357: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 13:27:41.094088: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 13:27:41.094168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 13:27:41.094839: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 13:27:41.101240: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 13:27:41.101983: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [ ]:
# Load all datasets from './data_sets'
data_path = './data_sets'
input_datasets = []
output_datasets = []

# Assuming your files are named input_data_1.csv, input_data_2.csv, ..., output_data_1.csv, output_data_2.csv, ...
for i in range(1, 501):
    input_file = os.path.join(data_path, f'input_data_{i}.csv')
    output_file = os.path.join(data_path, f'output_data_{i}.csv')

    # Load data using pandas
    input_data = pd.read_csv(input_file).values
    output_data = pd.read_csv(output_file).values

    input_datasets.append(input_data)
    output_datasets.append(output_data)

In [ ]:
# Convert the data to NumPy arrays
X = np.array(input_datasets, dtype=float)
y = np.array(output_datasets, dtype=float)

# Reshape y to be a 2D array
y = y.reshape(-1, 1)

# Normalize the input data
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)

# Normalize the output data
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a simple LSTM regression model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))  # Output layer with one neuron for regression
model.compile(optimizer='adam', loss='mse')  # Mean Squared Error loss for regression

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Mean Squared Error on Test Set: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions to original scale
predictions = scaler_y.inverse_transform(predictions)

# Print the predictions
print('Predictions:')
print(predictions)